In [1]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x1825e945ee0>)

In [2]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 21367

sentences = []
labels = []

In [3]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [4]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [5]:
print(len(testing_sentences))

5342


In [6]:
print(len(training_sentences))

21367


In [7]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [8]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(256, activation='selu'),
        tf.keras.layers.Dense(128, activation='selu'),
        tf.keras.layers.Dense(64, activation='selu'),
        tf.keras.layers.Dense(16, activation='selu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs,
              validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/30
668/668 [==============================] - 31s 37ms/step - loss: 0.5051 - accuracy: 0.7395 - val_loss: 0.4018 - val_accuracy: 0.8147
Epoch 2/30
668/668 [==============================] - 23s 34ms/step - loss: 0.3887 - accuracy: 0.8189 - val_loss: 0.4022 - val_accuracy: 0.8139
Epoch 3/30
668/668 [==============================] - 22s 34ms/step - loss: 0.3605 - accuracy: 0.8344 - val_loss: 0.3847 - val_accuracy: 0.8225
Epoch 4/30
668/668 [==============================] - 23s 34ms/step - loss: 0.3420 - accuracy: 0.8456 - val_loss: 0.3757 - val_accuracy: 0.8272
Epoch 5/30
668/668 [==============================] - 23s 35ms/step - loss: 0.3218 - accuracy: 0.8552 - val_loss: 0.3988 - val_accuracy: 0.8143
Epoch 6/30
668/668 [==============================] - 24s 35ms/step - loss: 0.3051 - accuracy: 0.8635 - val_loss: 0.3784 - val_accuracy: 0.8300
Epoch 7/30
668/668 [==============================] - 23s 35ms/step - loss: 0.2883 - accuracy: 0.8717 - val_loss: 0.4174 - val_accuracy: